<a href="https://colab.research.google.com/github/Strugoeden121/swinAutoDecoder/blob/main/TransformerDecoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Install dependencies (if necessary)
!pip install torch torchvision
!pip install tqdm

from typing import Optional
import torch
from torch import Tensor
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision import datasets, models

# Step 2: Dataset Preparation
class ImageSequenceDataset(Dataset):
    """Dataset for image sequences"""
    def __init__(self, transform=None, seq_len=10):
        self.seq_len = seq_len
        self.transform = transform
        # Using MNIST dataset here (you can replace it with other datasets)
        self.data = datasets.MNIST(root='./data', download=True, train=True)

    def __len__(self):
        return len(self.data) - self.seq_len

    def __getitem__(self, idx):
        frames = []
        for i in range(self.seq_len):
            img, _ = self.data[idx + i]
            if self.transform:
                img = self.transform(img)
            frames.append(img)

        frames = torch.stack(frames)  # (seq_len, channels, height, width)
        return frames, frames[-1]  # Using target sequence as shifted sequence for simplicity

# Step 3: CNN Encoder (to extract features from each frame)
class CNNEncoder(nn.Module):
    def __init__(self, input_channels=1, feature_dim=256):  # Set input_channels to 3 for CIFAR-10
        super(CNNEncoder, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, feature_dim, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc = nn.Linear(feature_dim * 3 * 3, feature_dim)  # Adjust based on output size after pooling

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc(x)  # (batch_size, feature_dim)
        return x


# Step 4: Causal Transformer Decoder Layer and Decoder
class CausalTransformerDecoderLayer(nn.TransformerDecoderLayer):
    def forward(
        self,
        tgt: Tensor,
        memory: Optional[Tensor] = None,
        memory_mask: Optional[Tensor] = None,
        tgt_key_padding_mask: Optional[Tensor] = None,
        memory_key_padding_mask: Optional[Tensor] = None,
    ) -> Tensor:
        if self.training:
            return super().forward(
                tgt,
                memory,
                tgt_mask=generate_square_subsequent_mask(tgt.size(0), tgt.device),
                memory_mask=memory_mask,
                tgt_key_padding_mask=tgt_key_padding_mask,
                memory_key_padding_mask=memory_key_padding_mask,
            )

        tgt_last_tok = tgt[-1:, :, :]
        tmp_tgt = self.self_attn(
            tgt_last_tok,
            tgt,
            tgt,
            attn_mask=None,
            key_padding_mask=tgt_key_padding_mask,
        )[0]
        tgt_last_tok = tgt_last_tok + self.dropout1(tmp_tgt)
        tgt_last_tok = self.norm1(tgt_last_tok)

        if memory is not None:
            tmp_tgt = self.multihead_attn(
                tgt_last_tok,
                memory,
                memory,
                attn_mask=memory_mask,
                key_padding_mask=memory_key_padding_mask,
            )[0]
            tgt_last_tok = tgt_last_tok + self.dropout2(tmp_tgt)
            tgt_last_tok = self.norm2(tgt_last_tok)

        tmp_tgt = self.linear2(self.dropout(self.activation(self.linear1(tgt_last_tok))))
        tgt_last_tok = tgt_last_tok + self.dropout3(tmp_tgt)
        tgt_last_tok = self.norm3(tgt_last_tok)

        return tgt_last_tok

class CausalTransformerDecoder(nn.TransformerDecoder):
    def forward(
        self,
        tgt: Tensor,
        memory: Optional[Tensor] = None,
        cache: Optional[Tensor] = None,
        memory_mask: Optional[Tensor] = None,
        tgt_key_padding_mask: Optional[Tensor] = None,
        memory_key_padding_mask: Optional[Tensor] = None,
    ) -> Tensor:
        output = tgt
        if self.training:
            if cache is not None:
                raise ValueError("cache parameter should be None in training mode")
            for mod in self.layers:
                output = mod(output, memory, memory_mask=memory_mask, tgt_key_padding_mask=tgt_key_padding_mask)
            return output

        new_token_cache = []
        for i, mod in enumerate(self.layers):
            output = mod(output, memory)
            new_token_cache.append(output)
            if cache is not None:
                output = torch.cat([cache[i], output], dim=0)

        if cache is not None:
            new_cache = torch.cat([cache, torch.stack(new_token_cache, dim=0)], dim=1)
        else:
            new_cache = torch.stack(new_token_cache, dim=0)

        return output, new_cache

def generate_square_subsequent_mask(sz: int, device: str = "cpu") -> torch.Tensor:
    mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float("-inf")).masked_fill(mask == 1, float(0.0))
    return mask.to(device=device)

# Step 5: Training Setup
class ImageSequenceModel(nn.Module):
    def __init__(self, seq_len=10, feature_dim=256, output_dim=256):
        super(ImageSequenceModel, self).__init__()
        self.encoder = CNNEncoder(input_channels=1, feature_dim=feature_dim)  # CIFAR-10 has 3 channels
        self.decoder_layer = CausalTransformerDecoderLayer(d_model=feature_dim, nhead=4)
        self.decoder = CausalTransformerDecoder(self.decoder_layer, num_layers=6)
        self.fc_out = nn.Linear(feature_dim, 1 * 28 * 28)  # Output should match CIFAR-10 size

    def forward(self, x):
        batch_size, seq_len, C, H, W = x.size()
        memory = []
        for i in range(seq_len):
            frame_features = self.encoder(x[:, i, :, :, :])
            memory.append(frame_features)
        memory = torch.stack(memory, dim=0)  # (seq_len, batch_size, feature_dim)

        tgt = memory  # Use memory as the target as well for simplicity
        output = self.decoder(tgt, memory)[0]  # (seq_len, batch_size, feature_dim)

        # Flatten and map to output size (batch_size, seq_len, 3, 28, 28)
        output = self.fc_out(output)  # (seq_len, batch_size, 3 * 28 * 28)
        output = output.view(batch_size, 1, 28, 28)  # Reshape to (seq_len, batch_size, 3, 28, 28)

        return output
from tqdm import tqdm

# Step 6: Training Loop with Debugging Prints and tqdm
def train(model, dataloader, criterion, optimizer, epochs=10):
    model.train()
    print("started training")
    for epoch in range(epochs):
        running_loss = 0.0
        # Adding tqdm for batch progress
        for i, (inputs, targets) in enumerate(tqdm(dataloader, desc=f"Epoch {epoch+1}", leave=False)):
            # Debugging prints
            print(f"Epoch {epoch+1}, Batch {i+1}/{len(dataloader)}")

            optimizer.zero_grad()
            outputs = model(inputs)

            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {running_loss/len(dataloader)}")

# Step 7: Testing Loop with Debugging Prints and tqdm
def test(model, dataloader):
    model.eval()
    print("started testing")
    with torch.no_grad():
        total_loss = 0.0
        # Adding tqdm for batch progress
        for i, (inputs, targets) in enumerate(tqdm(dataloader, desc="Testing", leave=False)):
            # Debugging prints
            print(f"Test Batch {i+1}/{len(dataloader)}")

            outputs = model(inputs)

            # Debugging prints for output shape

            loss = nn.MSELoss()(outputs, targets)
            total_loss += loss.item()

        print(f"Test Loss: {total_loss / len(dataloader)}")

# Step 8: Main Setup (for Google Colab)
if __name__ == "__main__":
    # Data augmentation and transformation for the dataset
    transform = transforms.Compose([transforms.Resize((28, 28)), transforms.ToTensor()])

    # Load the MNIST dataset (can be replaced with a custom dataset for sequences)
    dataset = ImageSequenceDataset(transform=transform, seq_len=10)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

    # Initialize model, loss function, and optimizer
    model = ImageSequenceModel(seq_len=10, feature_dim=256, output_dim=256)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    # Train and test the model
    print("Starting training...")
    train(model, dataloader, criterion, optimizer, epochs=10)
    print("Training complete. Starting testing...")
    test(model, dataloader)


# Step 8: Main Setup (for Google Colab)
if __name__ == "__main__":
    # Data augmentation and transformation for the dataset
    transform = transforms.Compose([transforms.Resize((28, 28)), transforms.ToTensor()])

    # Load the MNIST dataset (can be replaced with a custom dataset for sequences)
    dataset = ImageSequenceDataset(transform=transform, seq_len=10)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

    # Initialize model, loss function, and optimizer
    model = ImageSequenceModel(seq_len=10, feature_dim=256, output_dim=256)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    # Train and test the model
    train(model, dataloader, criterion, optimizer, epochs=10)
    test(model, dataloader)


Starting training...
started training


Epoch 1:   0%|          | 0/1875 [00:00<?, ?it/s]

Epoch 1, Batch 1/1875


Epoch 1:   0%|          | 1/1875 [00:00<24:04,  1.30it/s]

Epoch 1, Batch 2/1875


...

Epoch 1:  22%|██▏       | 405/1875 [05:11<17:32,  1.40it/s]

Epoch 1, Batch 406/1875


Epoch 1:  22%|██▏       | 406/1875 [05:12<17:25,  1.40it/s]

Epoch 1, Batch 407/1875


Epoch 1:  22%|██▏       | 407/1875 [05:13<17:26,  1.40it/s]